In [ ]:
# Some useful settings for interactive work
%load_ext autoreload
%autoreload 2

%matplotlib widget

import torch
torch.set_float32_matmul_precision('high')

In [ ]:
# Import the relevant modules
import sousvide.synthesize.rollout_generator as rg
import sousvide.synthesize.observation_generator as og
import sousvide.instruct.train_policy as tp
import sousvide.visualize.plot_synthesize as ps
import sousvide.visualize.plot_learning as pl
import sousvide.flight.deploy_figs as df

In [97]:
import numpy as np
import figs.utilities.config_helper as ch
import figs.utilities.transform_helper as th
from figs.control.vehicle_rate_fqp import VehicleRateFQP
import sousvide.visualize.plot_time as pt
import sousvide.visualize.plot_3D as p3d
import matplotlib.pyplot as plt

np.random.seed(0)
mu_md = np.zeros(10)
std_md = np.array([
            0.40, 0.40, 0.40,
            0.40, 0.40, 0.40,
            0.20, 0.20, 0.20, 0.20
            ])

# policy = ch.get_config("vrfqp_fr_v1","pilots")
policy = ch.get_config("vrfqp_fr_v2","pilots")
# course = ch.get_config("robustness_track","courses")
course = ch.get_config("bounce1","courses")
# course = ch.get_config("bounce2","courses")

test = VehicleRateFQP(policy,course)
# p3d.tXU_to_3D([test.tXUd])

Optimization terminated successfully    (Exit mode 0)
            Current function value: 18791.624123630354
            Iterations: 14
            Function evaluations: 67
            Gradient evaluations: 14
4.927923553847117 [0.         1.58756497 3.34035859]


In [98]:
# policy = ch.get_config("vrfqp_fr_v1","pilots")
policy = ch.get_config("vrfqp_fr_v2","pilots")
# course = ch.get_config("robustness_track","courses")
# course = ch.get_config("bounce1","courses")
course = ch.get_config("bounce2","courses")

test = VehicleRateFQP(policy,course)
# p3d.tXU_to_3D([test.tXUd])

Optimization terminated successfully    (Exit mode 0)
            Current function value: 18630.56909158912
            Iterations: 14
            Function evaluations: 88
            Gradient evaluations: 14
5.006345291123313 [0.         1.69482546 3.31151983]
